# Measuring the business impact of your recommendations

This notebook will walk you through an example of using [metric attributions](https://docs.aws.amazon.com/personalize/latest/dg/metric-attribution-requirements.html) in [Amazon Personalize](https://aws.amazon.com/personalize/) to measure how your personalized recommendaations are helping you achieve your business goals. 

Personalized recommendations increase user engagement with your websites or apps. If you want to meassure the impact of your recommendations on your KPIs [Amazon Personalize metric attributions](https://docs.aws.amazon.com/personalize/latest/dg/UPDATE) can help. For example, a video on demand application may want to see the impact of recommendations on click through rate (CTR) or total time watched. A retail application may want to track a click-through rates and margins.

![iagram showing how metrics from user interactions can be tracked and exported to Amazon S3 or Amazon CloudWatch](images/metrics-overview.png "Diagram showing how metrics from user interactions can be tracked and exported to Amazon S3 or Amazon CloudWatch")

Figure 1. Feature Overview. The interactions dataset is used to train a recommender or campaign. Then, when users interact with recommended items, these interactions are sent to Amazon Personalize and attributed to the corresponding recommender or campaign. These metrics are then exported to Amazon S3 and Amazon CloudWatch so you can monitor them and compare the metrics of each recommender or campaign.

![Diagram showing how metrics from user interactions with two scenarios can be tracked and exported to Amazon S3 or Amazon CloudWatch](images/metrics-overview-scenarios.png "Diagram showing how metrics from user interactions with two scenarios can be tracked and exported to Amazon S3 or Amazon CloudWatch")

Figure 2. Measuring the business impact of recommendations in two scenarios. The interactions dataset is used to train two recommenders or campaigns, in this case designated “Blue” and “Orange”. Then, when users interact with recommended items, these interactions are sent to Amazon Personalize and attributed to the corresponding recommender, campaign, or scenario the user was exposed to when they interacted with the item. These metrics are then exported to Amazon S3 and Amazon CloudWatch so you can monitor them and compare the metrics of each recommender or campaign.

First, we'll follow the steps to build a Domain dataset group. Then we will create the metrics attribution to track two metrics:

1. the click-though rate for recommendations 
1. the total margin from purchases

Then we will upload synthetic data based on a data set generated for [a ficticious retail store](https://github.com/aws-samples/retail-demo-store). We will we will train a recommender that returns product recommendations. 

We will then interact with this recommender and and see how metrics are exported to Amazon S3.

The goal is to recommend products that are relevant for each particular user and to measure the impact of the recommendations.

Finally, we'll cleanup all of the resources we created so we avoid incurring costs for resources that are no longer being used using the notebook [Clean_Up_Resources.ipynb](Clean_Up_Resources.ipynb).  

The estimated time to run through this notebook is about 40 minutes.  

## How to use the Notebook

The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started.

## Introduction to Amazon Personalize

[Amazon Personalize](https://aws.amazon.com/personalize/) makes it easy for customers to develop applications with a wide array of personalization use cases, including real time product recommendations and customized direct marketing. Amazon Personalize brings the same machine learning technology used by Amazon.com to everyone for use in their applications – with no machine learning experience required. Amazon Personalize customers pay for what they use, with no minimum fees or upfront commitment. 

You can start using Amazon Personalize with a simple three step process, which only takes a few clicks in the AWS console, or a set of simple API calls. 

* First, point Amazon Personalize to user data, catalog data, and activity stream of views, clicks, purchases, etc. in Amazon S3 or upload using a simple API call. 

* Second, with a single click in the console or an API call, train a private recommendation model for your data. 

* Third, retrieve personalized recommendations for any user by creating a recommender, and using the GetRecommendations API.

If you are not familiar with Amazon Personalize, you can learn more about the service on by looking at [Github Sample Notebooks](https://github.com/aws-samples/amazon-personalize-samples) and [Product Documentation](https://docs.aws.amazon.com/personalize/latest/dg/what-is-personalize.html).

## Imports
Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) which are core data science tools.

In [ ]:
# Get the latest version of botocore to ensure we have the latest features in the SDK
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as md

## Specify the AWS region we will use
This is the region where the Dataset Group will be Created

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# or you can set your own region using:
# region = "us-west-2"

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [ ]:
personalize.list_dataset_groups()

Setup names for items and interactions files to use later

In [ ]:
interactions_file_path = 'cleaned_interactions_training_data.csv'
items_file_path = 'cleaned_item_training_data.csv'

## Create the Amazon Personalize role

Amazon Personalize needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it so it can access data from S3. The code below attaches very permissive policies, please use more restrictive policies for any production application.

Note: Make sure the role you are using to run the code in this notebook has the necessary permissions to create a role.

In [ ]:
iam = boto3.Session().client(
    service_name='iam', region_name=region)
account_id = boto3.Session().client(
    service_name='sts', region_name=region).get_caller_identity().get('Account')

In [ ]:
role_name = account_id+"measure-impact-recommendations"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "personalize.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create or retrieve the role:
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );
    role_arn = create_role_response["Role"]["Arn"]
    
except iam.exceptions.EntityAlreadyExistsException as e:
    print('Warning: role already exists: {}\n'.format(e))
    role_arn = iam.get_role(
        RoleName = role_name
    )["Role"]["Arn"];

print('IAM Role: {}\n'.format(role_arn))

## Attach the policy if it is not previously attached

In [ ]:
policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess" #Allows access to any S3 bucket, limit as required

if (policy_arn in [ x['PolicyArn'] for x in iam.list_attached_role_policies( RoleName = role_name)['AttachedPolicies']]):
    print ('The policy {} is already attached to this role.'.format(policy_arn))
else:
    print ("Attaching the role_policy")
    attach_response = iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
    );
    print ("30s pause to allow role to be fully consistent.")
    time.sleep(30)
    print('Done.')
    
    
access_cloudwatch_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricData"
            ],
            "Resource":"*"
        }
    ]
}

print ("Attaching the role_policy to Access Cloudwatch")
response = iam.put_role_policy(
    RoleName = role_name,
    PolicyName='AccesCloudwatch',
    PolicyDocument=json.dumps(access_cloudwatch_role_policy_document)
)

print (response)

## Specify an S3 Bucket for your data

Amazon Personalize will need an S3 bucket to act as the source of your data. The code bellow will create a bucket with a unique `bucket_name`. We will aso use this bucket to export data.

The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources. 

In [ ]:
s3 = boto3.Session().client(
    service_name='s3', region_name=region)

In [ ]:
bucket_name = account_id + "-" + region + "-measure-impact-recommendations"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists. Using bucket", bucket_name)

### Update the bucket policy

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "personalize.amazonaws.com",
                ]
            },
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

bucket_current_policy = None

try:
    bucket_current_policy = s3.get_bucket_policy(Bucket=bucket_name)['Policy']
    
except s3.exceptions.from_code('NoSuchBucketPolicy') as e:    
    print("There is no current Bucket Policy for bucket " + bucket_name)
    
except Exception as e: 
    raise(e)

if (bucket_current_policy and policy == json.loads(bucket_current_policy)):
    print ("The policy is already associated with the S3 Bucket.")
else:
    print ("Adding the policy to the bucket.")
    print(s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy)))

## Download, Prepare, and Upload Training Data

We generated the synthetic data based on the code in the [Retail Demo Store project](https://github.com/aws-samples/retail-demo-store). Follow the link to learn more about the data and potential uses.

First we need to download the data (training data). In this tutorial we'll use the Purchase history from a retail store  dataset. 

### Download and Explore the Interactions Dataset

In [ ]:
!wget https://code.retaildemostore.retail.aws.dev/csvs/interactions.csv

The dataset has been successfully downloaded as interactions.csv

Let's learn more about the dataset by viewing its charateristics:

In [ ]:
interaction_data = pd.read_csv('./interactions.csv')
interaction_data

In [ ]:
interaction_data.info()

The ECOMMERCE recommenders require you to provide specific EVENT_TYPE values in order to understand the context of an interaction. Let's look at what event types are currently in our dataset:

In [ ]:
interaction_data.EVENT_TYPE.value_counts()

From the cells above, we've learned that our data has has 5 columns, 675004 rows and the headers are: ITEM_ID, USER_ID, EVENT_TYPE, TIMESTAMP and DISCOUNT.

The rows look like what we expect, we have:

* item_id of the product hte user interacted with
* user_id of the user who did the interaction
* event_type that defines the interaction. The event type can be View, AddToCart, StartCheckout, or Purchase
* timestamp in unix format
* wether a discount was applied

To be compatible with an Amazon Personalize interactions schema, this dataset requires column headings compatible with Amazon Personalize default column names (read about column names [here](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html) )


### Prepare the Interactions Data


#### Drop Columns

Some columns in this dataset would not add value to our model and as such need to be dropped from this dataset. Columns such as *discount*.

In [ ]:
interaction_data.drop(columns=['DISCOUNT'], inplace = True)

interaction_data.head()

We can see that 'View' and 'Purchase' event are present and we can proceed. 

In [ ]:
interaction_data.to_csv(interactions_file_path)

In the cell below, we will write our cleaned data to a file named 'cleaned_interactions_training_data.csv'

### Upload Interactions data to S3
Now that our training data is ready for Amazon Personalize,the next step is to upload it to the s3 bucket created earlier

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path

### Download and Explore the Items Dataset

In [ ]:
!wget https://code.retaildemostore.retail.aws.dev/csvs/items.csv

The dataset has been successfully downloaded as items.csv

Lets learn more about the dataset by viewing its charateristics

In [ ]:
items_df = pd.read_csv('./items.csv')
items_df

In [ ]:
items_df.info()

Let's explore the kinds of items included in the dataset.

In [ ]:
items_df.CATEGORY_L1.unique()

In [ ]:
items_df.CATEGORY_L2.unique()

### Drop Columns

Some columns in this dataset could add value to our model but are not relevant for this example. For simplicity, we will drop them from this dataset. Columns such as *product_decription*.

In [ ]:
items_df.drop(columns=['PRODUCT_DESCRIPTION'], inplace = True)
items_df.head()

Write our cleaned data to a .csv file 

In [ ]:
items_df.to_csv(items_file_path)

#### Add a margin column

We will add an additional column to represent the margin. In this example we will use a randomly generated number between 0 and 10 to represent the item revenue margin. We will add this column.

In [ ]:
random = np.random.rand( len(items_df.index))
items_df["MARGIN"] = random * 10
items_df.head()

In the cell below, we will write our cleaned data to a file named 'cleaned_item_training_data.csv'

In [ ]:
items_df.to_csv(items_file_path)

### Upload Items data to S3
Now that our training data is ready for Amazon Personalize,the next step is to upload it to the s3 bucket created earlier

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_file_path).upload_file(items_file_path)
items_s3DataPath = "s3://"+bucket_name+"/"+items_file_path

## Create the Dataset Group
The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, recommenders, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like. 

When you create a Domain dataset group, you choose your domain. The domain you specify determines the default schemas for datasets and the use cases that are available for recommenders. 

You can find more information about creating a Domain dataset group in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/create-domain-dataset-group.html).

### Create Dataset Group

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Wait for Dataset Group to Have ACTIVE Status
Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

## Create Interactions Schema
A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"  
        }
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interactions-schema",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Create Items Schema
A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "PRICE",
            "type": "float"
        },
        {
            "name": "CATEGORY_L1",
            "type": ["string"],
            "categorical": True
        },
        {
            "name": "CATEGORY_L2",
            "type": ["string"],
            "categorical": True
        },
        {
            "name": "MARGIN",
            "type": "double"    
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-item_group",
    domain = "ECOMMERCE",
    schema = json.dumps(items_schema)
)

items_schema_arn = create_schema_response['schemaArn']

print(json.dumps(create_schema_response, indent=2))

## Create Datasets
After the group, the next thing to create is the datasets where your data will be uploaded to in Amazon Personalize.

### Create Interactions Dataset

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Create Items Dataset

In [ ]:
dataset_type = "ITEMS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

Let's wait until all the datasets have been created.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = interactions_dataset_arn
    )
    status_interaction_dataset =  describe_dataset_response["dataset"]['status']
    print("Interactions Dataset: {}".format(status_interaction_dataset))
    
    if status_interaction_dataset == "ACTIVE":
        print("Build succeeded for {}".format(interactions_dataset_arn))
        
    elif status_interaction_dataset == "CREATE FAILED":
        print("Build failed for {}".format(interactions_dataset_arn))
        break
        
    if not status_interaction_dataset == "ACTIVE":
        print("The interaction dataset creation is still in progress")
    else:
        print("The interaction dataset  is ACTIVE")
        

    describe_dataset_response = personalize.describe_dataset(
        datasetArn = items_dataset_arn
    )
    status_item_dataset =  describe_dataset_response["dataset"]['status']
    print("Items Dataset: {}".format(status_item_dataset))
    
    if status_item_dataset == "ACTIVE":
        print("Build succeeded for {}".format(items_dataset_arn))
        
    elif status_item_dataset == "CREATE FAILED":
        print("Build failed for {}".format(items_dataset_arn))
        break
    if not status_item_dataset == "ACTIVE":
        print("The item dataset creation is still in progress")
    else:
        print("The item dataset  is ACTIVE")
    
    
    if status_interaction_dataset == "ACTIVE" and status_item_dataset == "ACTIVE":
        break
        
    time.sleep(30)    

## Create a metric attribution

Before importing our data to Amazon Personalize, we will define the metrics attribution. 

### Important
After you create a metric attribution, Amazon Personalize automatically sends metrics to CloudWatch. if you record events or import incremental bulk dara, you incur some monthly CloudWatch cost per metric. For information about CloudWatch oricing, see the [Amazon CloudWatch pricing page](https://aws.amazon.com/cloudwatch/pricing/). To stop sending metrics to CloudWatch, delete the metric attribution.

In [ ]:
metric_attribution_name = 'my-first-metric-attribution'

# If metrics attribution already exist, delete it
metric_attribution_list = personalize.list_metric_attributions(datasetGroupArn = dataset_group_arn)['metricAttributions']

for metric in metric_attribution_list:
    if (metric_attribution_name == metric['name']):
        response = personalize.delete_metric_attribution(
            metricAttributionArn = metric['metricAttributionArn']
        ) 
        print (response)
        #wait for metric to delete
        time.sleep (60) 

We will create two metric attributions, one to count the Views, and one to sum the margin each time an item is purchased. we will be using the two expressions `SAMPLECOUNT()` and `SUM(DatasetType.column_name)`. You can create up to 10 different attributions in a dataset group.

In [ ]:
metrics_list = [{ 
        "eventType": "View",
        "expression": "SAMPLECOUNT()",
        "metricName": "countViews"
    }
    ,
    {
        "eventType": "Purchase",
        "expression": "SUM(ITEMS.MARGIN)",
        "metricName": "sumMargin"
    }
]

output_config = {
  "roleArn": role_arn, 
  "s3DataDestination": {
    "path": "s3://"+bucket_name+"/metrics/"
  }
}

response = personalize.create_metric_attribution(
  name = metric_attribution_name,
  datasetGroupArn = dataset_group_arn,
  metricsOutputConfig = output_config,
  metrics = metrics_list
)

metric_attribution_arn = response['metricAttributionArn']

print ('Metric attribution ARN: ' + metric_attribution_arn)

description = personalize.describe_metric_attribution( 
    metricAttributionArn = metric_attribution_arn)['metricAttribution']

print('Name: ' + description['name'])
print('ARN: ' + description['metricAttributionArn'])
print('Status: ' + description['status'])

Let us wait for the metric attribution to be created.

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_metric_response = personalize.describe_metric_attribution(
        metricAttributionArn = metric_attribution_arn
    )
    status = describe_metric_response['metricAttribution']['status']
    print("Status: {}".format(status))

    if status == "ACTIVE" or status == 'CREATE FAILED':
        break

    time.sleep(60)

In [ ]:
personalize.list_metric_attributions()['metricAttributions']

## Import the data
Earlier you created the DatasetGroup and Dataset to house your information, now you will execute an import job that will load the data from S3 into Amazon Personalize for usage building your model.
### Create Interactions Dataset Import Job

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Create Items Dataset Import Job

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_items_import2",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_file_path)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### Wait for Dataset Import Jobs to Have ACTIVE Status
It can take a while before the import jobs complete, please wait until you see that they are active below.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("ItemsDatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("InteractionsDatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Choose a recommender use case

Each domain has different use cases. When you create a recommender you create it for a specific use case, and each use case has different requirements for getting recommendations.

In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

We are going to create 2 recommenders 

* [Recommended For You](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-use-cases.html#recommended-for-you-use-case). This type of recommender offers personalized recommendations for items based on a user that you specify. 
* [Customers who viewed X also viewed](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-use-cases.html#customers-also-viewed-use-case). This type of recommender offers personalized recommendations for items that customers also viewed based on an item that you specify. 

In both these use cases, Amazon Personalize automatically filters items the user purchased based on the userId that you specify and Purchase events.

[More use cases per domain](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

### Create the "Recommended For You" recommender

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

### Create the "Customers who viewed X also viewed" recommender

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_who_viewed_x',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  datasetGroupArn = dataset_group_arn
)
who_viewed_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

We wait until the recomenders have finished creating and have status `ACTIVE`. We check periodically on the status of the recommender

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('At least one recommender build is still in progress.')
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = who_viewed_x_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(who_viewed_x_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(who_viewed_x_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('At least one recommender build is still in progress.')
        
    time.sleep(60)

## Getting recommendations with a recommender
Now that the recommender has been trained, lets have a look at the recommendations we can get for our users and items!

In [ ]:
# reading the original data in order to have a dataframe that has both item_ids 
# and the corresponding titles to make our recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.head()

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

In [ ]:
def get_category_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item category.
    
    A really broad try/except clause was added in case anything goes wrong.
    """
    
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['CATEGORY_L2'].values[0]
    except:
        print (item_id)
        return "Error obtaining item category"
    

Let us get some  recommendations from the recommender returning "Recommended for you":

In [ ]:
# First pick a user
test_user_id = "777" 

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

### "Customers who viewed X also viewed" recommender
"Customers who viewed X also viewed" recommender requires an item and a user as input, and it will return customers also viewed based on an item that you specify.

The cells below will handle getting recommendations from the "Customers who viewed X also viewed" Recommender and rendering the results. Let's see what the recommendations are for an item.

We will be using the `recommenderArn`, the `itemId`, the `userId`, as well as the number or results we want, numResults.

#### Select a User
We'll just pick a random user for simplicity (there are about 6,000 users with user IDs assigned in sequential order). Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [ ]:
user_id = 555

We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [ ]:
product_id = items_df.sample(1)["ITEM_ID"].values[0]

print ('productId: ', product_id)

get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = who_viewed_x_arn,
    itemId = str(product_id),
    userId = str(user_id),
    numResults = 40
)
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_by_id( itm['itemId'], items_df) for itm in item_list ]
df['Category'] = [ get_category_by_id ( itm['itemId'], items_df) for itm in item_list ]
display (df)

This is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar items render in a nice dataframe. Let's create a helper function to achieve this.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_viewed_x(recommendations_df, item_id, user_id):
    # Get the item name
    original_item_name = get_item_by_id(item_id, items_df)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = who_viewed_x_arn,
        itemId = str(item_id),
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_by_id(item['itemId'], items_df)
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [original_item_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different items. Let's sample some data from our dataset to test our "Customers who viewed X also viewed" Recommender. Grab 5 random items from our dataframe.

In [ ]:
samples = items_df.sample(5)
samples

In [ ]:
viewed_x_also_viewed_recommendations_df = pd.DataFrame()
items = samples.ITEM_ID.tolist()

for item in items:
    viewed_x_also_viewed_recommendations_df = get_new_recommendations_df_viewed_x(viewed_x_also_viewed_recommendations_df, item, user_id)

viewed_x_also_viewed_recommendations_df.head(10)

## Sending Interaction Data to Amazon Personalize

For the purposes of this demo, we will simulate sending data to Amazon personalize.

### Create Personalize Event Tracker
Let's start by creating an event tracker for our dataset group.

In [ ]:
event_tracker_response = personalize.create_event_tracker(
    datasetGroupArn=dataset_group_arn,
    name='amazon-personalize-event-tracker'
)

event_tracker_arn = event_tracker_response['eventTrackerArn']
event_tracking_id = event_tracker_response['trackingId']

print('Event Tracker ARN: ' + event_tracker_arn)
print('Event Tracking ID: ' + event_tracking_id)

### Wait for Event Tracker Status to Become ACTIVE
The event tracker should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Send Events and attribute them to the "recommended for you" recommender

Let us generate some events to track. We will create events based on the recommendations by the "recommended for you" recommender for several users. 

We will generate events starting now and backdate them decreasing in 2 minute intervals with a random allocation of 'View' and 'Purchase' (80% and 20% respectively).

In [ ]:
test_users = [555, 100, 120, 444, 70, 82]

today = datetime.datetime.now()
minutes_in_interval = 2
interval_multiplier = 1

possible_interactions = ['View', 'Purchase']

for user_id in test_users:
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommended_for_you_arn,
        userId = str(user_id),
        numResults = 30,
    )
    item_list = get_recommendations_response['itemList']

    df = pd.DataFrame()
    df['Item'] = [ itm['itemId'] for itm in item_list ]
    df['Name'] = [ get_item_by_id( itm['itemId'], items_df) for itm in item_list ]
    df['Category'] = [ get_category_by_id ( itm['itemId'], items_df) for itm in item_list ]

    for itm in item_list:

        time_to_convert = today - datetime.timedelta(minutes = minutes_in_interval * interval_multiplier)
        timestamp = int(time.mktime(time_to_convert.timetuple()))

        event = {
            'eventId': str(uuid.uuid4()),
            'eventType': np.random.choice(possible_interactions, p = [0.8, 0.2]),
            'itemId': itm['itemId'],
            'metricAttribution': {"eventAttributionSource": "recommended_for_you" },
            'sentAt': timestamp
        }

        response = personalize_events.put_events(
            trackingId=event_tracking_id,
            userId = str(user_id),
            sessionId = str(uuid.uuid4()),
            eventList = [event]
        )
        interval_multiplier += 1


### Send Events and attribute them to the "Customers who viewed X also viewed" recommender
We will do the same with the "Customers who viewed X also viewed" recommender, and select 6 random products. We will create events to track based on the recommendations by the "Customers who viewed X also viewed" recommender for several users. 

We will generate events starting now and backdate them decreasing in 2 minute intervals with a random allocation of 'View' and 'Purchase' (80% and 20% respectively).

In [ ]:
test_products = [
    '9dbf7c1a-3936-42ee-8c36-a28e94a35265',
    '1d9ab420-01b5-4d5d-b7b9-43c10f4eac84',
    'f5be9f67-8def-405d-af7e-2abf6876277f',
    'e5e842c6-9d8e-4940-a245-a0f02a5552ad',
    '08b05a0d-8f0d-412a-a2b1-97aa6c08a03d',
    '447bc9e4-c176-4665-892e-2a0f47b3a582']

In [ ]:
# we reset the interval multiplier
interval_multiplier = 1

for product_id in test_products:
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = who_viewed_x_arn,
        itemId = str(product_id),
        userId = str(user_id),
        numResults = 30
    )
    item_list = get_recommendations_response['itemList']

    df = pd.DataFrame()
    df['Item'] = [ itm['itemId'] for itm in item_list ]
    df['Name'] = [ get_item_by_id( itm['itemId'], items_df) for itm in item_list ]
    df['Category'] = [ get_category_by_id ( itm['itemId'], items_df) for itm in item_list ]


    for itm in item_list:
        time_to_convert = today - datetime.timedelta(minutes = minutes_in_interval * interval_multiplier)
        timestamp = int(time.mktime(time_to_convert.timetuple()))

        event = {
            'eventId': str(uuid.uuid4()),
            'eventType': np.random.choice(possible_interactions, p = [0.8, 0.2]),
            'itemId': itm['itemId'],
            'metricAttribution': {"eventAttributionSource": "viewed_X_also_viewed" },
            'sentAt': timestamp
            }

        res = personalize_events.put_events(
            trackingId=event_tracking_id,
            userId = str(user_id),
            sessionId = str(uuid.uuid4()),
            eventList = [event]
        )
        
        interval_multiplier += 1

## Inspect data from Amazon CloudWatch

Now that we have sent events to Amazon Personalize using putEvents, we can inspect the business metrics by querying data from Amazon CloudWatch.

You can also see this data using the [Amazon Cloudwatch Console](https://aws.amazon.com/console/) under the 'AWS/Personalize' namespace.

<div class="alert alert-block alert-warning">
<b>Note:</b> Data in CloudWatch can take some time to update, please wait at least 15 minutes to view the metrics.
</div>


In [ ]:
# initialize the cloudwatch client
cloud_watch = boto3.Session().client(
    service_name='cloudwatch', region_name=region)

Let us see the metrics we created in Amazon CloudWatch:

In [ ]:
# List metrics through the pagination interface
paginator = cloud_watch.get_paginator('list_metrics')

for response in paginator.paginate(Namespace='AWS/Personalize'):
    r1 = response['Metrics']
    for met in r1:
        if (met['MetricName'] in ['SUMMARGIN', 'COUNTVIEWS']):
            print (met)

We can see one metric of each type `COUNTVIEWS` and `SUMMARGIN` for each of our scenarios `RECOMMENDED_FOR_YOU` and `VIEWED_X_ALSO_VIEWED`.

Next we will query claudwatch for the latest metrics. We will compare the metrics for the two scenarios.

In [ ]:
# Define the start and end_times in Unix format this is the timeframe we will query
start_time = int(time.mktime((today + datetime.timedelta(hours = -5)).timetuple()))
end_time = int(time.mktime(today.timetuple()))


In [ ]:
# Getting metrics data from CloudWatch
recommended_for_you_sum_response = cloud_watch.get_metric_data(
    StartTime=start_time,
    EndTime=end_time,
    MetricDataQueries= [{
                "Id": "q1",
                "MetricStat": {
                    "Metric": {
                        "Namespace": 'AWS/Personalize',
                        "MetricName": "SUMMARGIN",
                        "Dimensions": [
                            {
                                'Name': 'EventAttributionSource',
                                'Value': 'RECOMMENDED_FOR_YOU'
                            },
                            {
                                'Name': 'DatasetGroupArn', 
                                'Value': dataset_group_arn
                            }
                        ]
                    },
                    "Period": 900, #minimum value is 900s = 15min
                    "Stat":"Sum",
                },
            }
        ]
    )  

print(recommended_for_you_sum_response)

viewed_also_viewed_sum_response = cloud_watch.get_metric_data(
    StartTime=start_time,
    EndTime=end_time,
    MetricDataQueries= [{
                "Id": "q1",
                "MetricStat": {
                    "Metric": {
                        "Namespace": 'AWS/Personalize',
                        "MetricName": "SUMMARGIN",
                        "Dimensions": [
                            {
                                'Name': 'EventAttributionSource',
                                'Value': 'VIEWED_X_ALSO_VIEWED'
                            },
                            {
                                'Name': 'DatasetGroupArn', 
                                'Value': dataset_group_arn
                            }
                        ]
                    },
                    "Period": 900, #minimum value is 900s = 15min
                    "Stat":"Sum",
                },
            }
        ]
    )
print(viewed_also_viewed_sum_response)

In [ ]:
# Getting data from the 
recommended_for_you_count_response = cloud_watch.get_metric_data(
    StartTime=start_time,
    EndTime=end_time,
    MetricDataQueries= [{
                "Id": "q1",
                "MetricStat": {
                    "Metric": {
                        "Namespace": 'AWS/Personalize',
                        "MetricName": "COUNTVIEWS",
                        "Dimensions": [
                            {
                                'Name': 'EventAttributionSource',
                                'Value': 'RECOMMENDED_FOR_YOU'
                            },
                            {
                                'Name': 'DatasetGroupArn', 
                                'Value': dataset_group_arn
                            }
                        ]
                    },
                    "Period": 900, #minimum value is 900s = 15min
                    "Stat":"Sum",
                },
            }
        ]
    )
print(recommended_for_you_count_response)

viewed_also_viewed_count_response = cloud_watch.get_metric_data(
    StartTime=start_time,
    EndTime=end_time,
    MetricDataQueries= [{
                "Id": "q1",
                "MetricStat": {
                    "Metric": {
                        "Namespace": 'AWS/Personalize',
                        "MetricName": "COUNTVIEWS",
                        "Dimensions": [
                            {
                                'Name': 'EventAttributionSource',
                                'Value': 'VIEWED_X_ALSO_VIEWED'
                            },
                            {
                                'Name': 'DatasetGroupArn', 
                                'Value': dataset_group_arn
                            }
                        ]
                    },
                    "Period": 900, #minimum value is 900s = 15min
                    "Stat":"Sum",
                },
            }
        ]
    )

print(viewed_also_viewed_count_response)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md

plt.xticks( rotation=25 )
ax=plt.gca()

xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)

plt.plot (
    recommended_for_you_sum_response['MetricDataResults'][0]['Timestamps'], 
    recommended_for_you_sum_response['MetricDataResults'][0]['Values']
)
plt.plot (
    viewed_also_viewed_sum_response['MetricDataResults'][0]['Timestamps'], 
    viewed_also_viewed_sum_response['MetricDataResults'][0]['Values']
)

In [ ]:
plt.xticks( rotation=25 )
ax=plt.gca()

xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)

plt.plot (
    recommended_for_you_count_response['MetricDataResults'][0]['Timestamps'], 
    recommended_for_you_count_response['MetricDataResults'][0]['Values']
)
plt.plot (
    viewed_also_viewed_count_response['MetricDataResults'][0]['Timestamps'], 
    viewed_also_viewed_count_response['MetricDataResults'][0]['Values']
)

In this plot we can compare how the margin (because of the purchases done by the customers) differs based on the scenario they were interacting with. 
It can also help you understand which rails are more effective.

## Review
Using the codes above you have successfully trained a deep learning model to generate item recommendations based on prior user behavior. You have created a recommenders for a foundational use case and you have used metric attributions to extract data to evaluate your model on business metrics.

Going forward, you can adapt this code to create other recommenders and metric attributions.

If you are done with this sample, make sure to follow the steps in the next section to cleanup the resources created in this notebook.

## Cleanup Resources 
This section contains instructions on how to clean up the resources created in this notebook.

### Save resource information for cleanup:
There are a few values you will need for the next notebook, execute the cell below to store them so they can be used in the `Clean_Up_Resources.ipynb` notebook.

This will overwite any data stored for those variables and set them to the values specified in this notebook. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

### Run the cleanup notebook

Continue to [Clean_Up_Resources.ipynb](Clean_Up_Resources.ipynb) clean up resources.